# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 88 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.16317


extracting tarball to tmp_2210.16317...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.16319


extracting tarball to tmp_2210.16319...

...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2210.17389/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


/tmp/ipykernel_2249/4030337529.py:34: LatexWarning: 2210.17430 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2210.17434


extracting tarball to tmp_2210.17434...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.17434-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.17434) | **The Near Infrared Imager and Slitless Spectrograph for the James Webb  Space Telescope -- IV. Aperture Masking Interferometry**  |
|| Anand Sivaramakrishnan, et al. -- incl., <mark>Alexandra Z. Greenbaum</mark>, <mark>Michael R. Meyer</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *30 pages, 10 figures*|
|**Abstract**| The James Webb Space Telescope's Near Infrared Imager and Slitless Spectrograph (JWST-NIRISS) flies a 7-hole non-redundant mask (NRM), the first such interferometer in space, operating at 3-5 \micron~wavelengths, and a bright limit of $\simeq 4$ magnitudes in W2. We describe the NIRISS Aperture Masking Interferometry (AMI) mode to help potential observers understand its underlying principles, present some sample science cases, explain its operational observing strategies, indicate how AMI proposals can be developed with data simulations, and how AMI data can be analyzed. We also present key results from commissioning AMI. Since the allied Kernel Phase Imaging (KPI) technique benefits from AMI operational strategies, we also cover NIRISS KPI methods and analysis techniques, including a new user-friendly KPI pipeline. The NIRISS KPI bright limit is $\simeq 8$ W2 magnitudes. AMI (and KPI) achieve an inner working angle of $\sim 70$ mas that is well inside the $\sim 400$ mas NIRCam inner working angle for its circular occulter coronagraphs at comparable wavelengths. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16921-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16921) | **A Chemical Modelling Roadmap Linking Protoplanetary Disks and Exoplanet  Atmospheres**  |
|| <mark>Christian Eistrup</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *Invited Review for Special Issue of the American Chemical Society. 51 pages*|
|**Abstract**| [Abridged] This review paper discussed which chemical effects may be at play in a planet-forming disk midplane, which effects are relevant under different conditions, and which tools are available for modelling chemical kinetics in a disk midplane. The review goes on to discuss some important efforts in the planet formation modelling community to treat chemical evolution, and, vice versa, efforts in the chemical modelling community to implement more physical effects related to planet formation into the chemical modelling. The aim of this review is both to outline some concepts related to planet formation chemistry, but also to encourage, not just collaboration between the planet formation modelling community and the astrochemical community, but also assistance and guidance from one community to the other. Guidance, regarding which effects, out of many, might be more relevant than others under certain planet formation conditions, and regarding why certain included effects lead to certain important modelling outcomes. As the research fields of exoplanet atmospheres and protoplanetary disks near new frontiers in observational insights with upcoming facilities, developing appropriate modelling frameworks (including physical and chemical effects) is paramount to ultimately enable the linking of a chemically characterised exoplanet atmospheres to its formation history in its natal protoplanetary disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16317-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16317) | **The estimation of far-field wavefront error of tilt-to-length distortion  coupling in space-based gravitational wave detection**  |
|| Ya-Zheng Tao, Hong-Bo Jin, <mark>Yue-Liang Wu</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *13 pages, 7 figures*|
|**Abstract**| In space-based gravitational wave detection, the estimation of far-field wavefront error of the distorted beam is the precondition for the noise reduction. Zernike polynomials is used to describe the wavefront error of the transmitted distorted beam. The propagation of a laser beam between two telescope apertures is calculated numerically. Far-field wavefront error is estimated with the absolute height of the peak-to-valley phase deviation between distorted Gaussian beam and a reference distortion-free Gaussian beam. The results show the pointing jitter is strongly related to the wavefront error. Furthermore, when jitter decreases 10 times from 100 to 10 nrad, wavefront error reduces for more than an order of magnitude. In the analysis of multi-parameter minimization, the minimum of wavefront error tends to Z[5,3] Zernike in some parameter ranges. Some Zernikes have a strong correlation with wavefront error of the received beam. When the aperture diameter increases at Z[5,3] Zernike, wavefront error is not monotonic and has oscillation. Nevertheless, wavefront error almost remains constant with the arm length increasing from 10$^{-1}$ Mkm to 10$^3$ Mkm. When the arm length decreases for three orders of magnitude from 10$^{-1}$ Mkm to 10$^{-4}$ Mkm, wavefront error has only an order of magnitude increasing. In the range of 10$^{-4}$ Mkm to 10$^3$ Mkm, the lowest limit of the wavefront error is from 0.5 fm to 0.015 fm, at Z[5,3] Zernike and 10 nrad jitter. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.17334-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.17334) | **COSMOS2020: Identification of High-z Protocluster Candidates in COSMOS**  |
|| Malte Brinch, et al. -- incl., <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *52 pages, 32 figues, 18 tables, main text is 30 pages, appendix is 22 pages, to be published in ApJ*|
|**Abstract**| We conduct a systematic search for protocluster candidates at $z \geq 6$ in the COSMOS field using the recently released COSMOS2020 source catalog. We select galaxies using a number of selection criteria to obtain a sample of galaxies that have a high probability of being inside a given redshift bin. We then apply overdensity analysis to the bins using two density estimators, a Weighted Adaptive Kernel Estimator and a Weighted Voronoi Tessellation Estimator. We have found 15 significant ($>4\sigma$) candidate galaxy overdensities across the redshift range $6\le z\le7.7$. The majority of the galaxies appear to be on the galaxy main sequence at their respective epochs. We use multiple stellar-mass-to-halo-mass conversion methods to obtain a range of dark matter halo mass estimates for the overdensities in the range of $\sim10^{11-13}\,M_{\rm \odot}$, at the respective redshifts of the overdensities. The number and the masses of the halos associated with our protocluster candidates are consistent with what is expected from the area of a COSMOS-like survey in a standard $\Lambda$CDM cosmology. Through comparison with simulation, we expect that all the overdensities at $z\simeq6$ will evolve into a Virgo-/Coma-like clusters at present (i.e., with masses $\sim 10^{14}-10^{15}\,M_{\rm \odot}$). Compared to other overdensities identified at $z \geq 6$ via narrow-band selection techniques, the overdensities presented appear to have $\sim10\times$ higher stellar masses and star-formation rates. We compare the evolution in the total star-formation rate and stellar mass content of the protocluster candidates across the redshift range $6\le z\le7.7$ and find agreement with the total average star-formation rate from simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.17389-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.17389) | **Zonal winds of Uranus and Neptune: Gravitational harmonics, dynamic  self-gravity, shape, and rotation**  |
|| Deniz Soyuer, Benno Neuenschwander, <mark>Ravit Helled</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| Uranus and Neptune exhibit fast surface zonal winds that can reach up to few hundred meters per second. Previous studies on zonal gravitational harmonics and Ohmic dissipation constraints suggest that the wind speeds diminish rapidly in relatively shallow depths within the planets. Through a case-by-case comparison between the missing dynamical gravitational harmonic $J^\prime_4$ from structure models, and with that expected from fluid perturbations, we put constraints on zonal wind decay in Uranus and Neptune. To this end, we generate polytropic empirical structure models of Uranus and Neptune using $4^{\rm th}$-order Theory of Figures (ToF) that leave hydrostatic $J_4$ as an open parameter. Allotting the missing dynamical contribution to density perturbations caused by zonal winds (and their dynamic self-gravity), we find that the maximum scale height of zonal winds are $\sim 2-3\%$ of the planetary radii for both planets. Allowing the models to have $J_2$ solutions in the $\pm 5 \times 10^{-6}$ range around the observed value has similar implications. The effect of self-gravity on $J^\prime_4$ is roughly a factor of ten lower than that of zonal winds, as expected. The decay scale heights are virtually insensitive to the proposed modifications to the bulk rotation periods of Uranus and Neptune in the literature. Additionally, we find that the dynamical density perturbations due to zonal winds have a measurable impact on the shape of the planet, and could potentially be used to infer wind decay and bulk rotation period via future observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.16322-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.16322) | **The Orbits and Dynamical Masses of the Castor System**  |
|| <mark>Guillermo Torres</mark>, et al. -- incl., <mark>Claire L. Davies</mark> |
|*Appeared on*| *2022-11-01*|
|*Comments*| *17 pages in emulateapj format, including figures and tables. Accepted for publication in The Astrophysical Journal*|
|**Abstract**| Castor is a system of six stars in which the two brighter objects, Castor A and B, revolve around each other every $\sim$450 yr and are both short-period spectroscopic binaries. They are attended by the more distant Castor C, which is also a binary. Here we report interferometric observations with the CHARA array that spatially resolve the companions in Castor A and B for the first time. We complement these observations with new radial velocity measurements of A and B spanning 30 yr, with the Hipparcos intermediate data, and with existing astrometric observations of the visual AB pair obtained over the past three centuries. We perform a joint orbital solution to solve simultaneously for the three-dimensional orbits of Castor A and B as well as the AB orbit. We find that they are far from being coplanar: the orbit of A is nearly at right angles (92 degrees) relative to the wide orbit, and that of B is inclined about 59 degrees compared to AB. We determine the dynamical masses of the four stars in Castor A and B to a precision better than 1%. We also determine the radii of the primary stars of both subsystems from their angular diameters measured with CHARA, and use them together with stellar evolution models to infer an age for the system of 290 Myr. The new knowledge of the orbits enables us to measure the slow motion of Castor C as well, which may assist future studies of the dynamical evolution of this remarkable sextuple system. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.17268-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.17268) | **Visual Brightness Characteristics of Starlink Generation 1 Satellites**  |
|| <mark>Anthony Mallama</mark>, Jay Respler |
|*Appeared on*| *2022-11-01*|
|*Comments*| **|
|**Abstract**| A large dataset of visual magnitudes for all three designs of Starlink satellites is analyzed. Brightness phase functions are derived for the Original, VisorSat and Post-VisorSat models. Similarities and differences between the functions for these spacecraft are noted. A metric called the characteristic magnitude is defined as the average brightness of a satellite when seen overhead at the end of astronomical twilight. When the phase functions are evaluated according to this metric, the characteristic magnitudes are: Original, 4.7; VisorSat, 6.2; and Post-VisorSat, 5.5. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.17430-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.17430) | **Unstructured Grid Dynamical Modeling of Planetary Atmospheres using  planetMPAS: The Influence of the Rigid Lid, Computational Efficiency, and  Examples of Martian and Jovian Application**  |
|| <mark>Yuan Lian</mark>, Mark I. Richardson |
|*Appeared on*| *2022-11-01*|
|*Comments*| *Manuscript has 61 pages, 20 figures, 2 tables, submitted to Planetary and Space Science*|
|**Abstract**| We present a new planetary global circulation model, planetMPAS, based on the state-of-the-art NCAR MPAS General Circulation Model. Taking advantage of the cross compatibility between WRF and MPAS, planetMPAS includes most of the planetWRF physics parameterization schemes for terrestrial planets such as Mars and Titan. PlanetMPAS also includes a set of physics that represents radiative transfer, dry convection, moist convection and its associated microphysics for the Jovian atmosphere. We demonstrate that, despite the rigid-lid approximation, planetMPAS is suitable to simulate the climate systems in Martian and Jovian atmospheres with potential application to slow-rotating planets such as Titan. Simulations using planetMPAS show that the new model can reproduce many aspects of the observed features on Mars and Jupiter, such as the seasonal CO2 cycle, polar argon enrichment, zonal mean temperature, and qualitative dust opacity on Mars, as well as the equatorial superrotation and banded zonal wind patterns on Jupiter. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.17434.md
    + _build/html/tmp_2210.17434/./Figures/ami_psf.png
    + _build/html/tmp_2210.17434/./Figures/NRMprototype.png
    + _build/html/tmp_2210.17434/./Figures/CLEARP.png
    + _build/html/tmp_2210.17434/./Figures/zodi_visamps_com_sim_rc.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\etal}{\textit{et al}.}$
$\newcommand{\ie}{\textit{i}.\textit{e}.}$
$\newcommand{\eg}{\textit{e}.\textit{g}.}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\nicemicron}{\text{\textmu m}}$
$\newcommand{\angstrom}{\textup{\AA}}$
$\newcommand{\uvplane}{uv-plane}$
$\newcommand{\mirage}{\texttt{MIRaGe}}$
$\newcommand{\finished}{\bf \color{black}}$
$\newcommand{\indraft}{\color{blue}}$
$\newcommand{\notstarted}{\color{red}}$
$\newcommand{\logl}{\ln \mathcal{L}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\etal$}{\textit{et al}.}$
$\newcommand{$\ie$}{\textit{i}.\textit{e}.}$
$\newcommand{$\eg$}{\textit{e}.\textit{g}.}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\nicemicron$}{\text{\textmu m}}$
$\newcommand{$\angstrom$}{\textup{\AA}}$
$\newcommand{$\uvplane$}{uv-plane}$
$\newcommand{$\mirage$}{\texttt{MIRaGe}}$
$\newcommand{$\finished$}{\bf \color{black}}$
$\newcommand{$\indraft$}{\color{blue}}$
$\newcommand{$\notstarted$}{\color{red}}$
$\newcommand{$\logl$}{\ln \mathcal{L}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# The Near Infrared Imager and Slitless Spectrograph for    the  James Webb Space Telescope -    IV. Aperture Masking Interferometry

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.17434-b31b1b.svg)](https://arxiv.org/abs/2210.17434)<mark>Appeared on: 2022-11-01</mark> - _30 pages, 10 figures_

</div>
<div id="authors">

Anand Sivaramakrishnan, et al. -- incl., <mark>Alexandra Z. Greenbaum</mark>, <mark>Michael R. Meyer</mark>

</div>
<div id="abstract">

**Abstract:** The James Webb Space Telescope's Near Infrared Imager and Slitless Spectrograph (JWST-NIRISS) flies a 7-hole non-redundant mask (NRM), the first such interferometer in space, operating at 3-5$\nicemicron$wavelengths, and a bright limit of$\simeq 4$magnitudes in W2.   We describe the NIRISS Aperture Masking Interferometry (AMI) mode to help potential observers understand its underlying principles, present some sample science cases, explain its operational observing strategies, indicate how AMI proposals can be developed with data simulations, and how AMI data can be analyzed.  We also present key results from commissioning AMI.  Since the allied Kernel Phase Imaging (KPI) technique benefits from AMI operational strategies, we also cover NIRISS KPI methods and analysis techniques, including a new user-friendly KPI pipeline.  The NIRISS KPI bright limit is$\simeq 8$W2 magnitudes.  AMI (and KPI) achieve an inner working angle of$\sim  70$mas that is well inside the$\sim 400$mas NIRCam inner working angle for its circular occulter coronagraphs at comparable wavelengths.

</div>

<div id="div_fig1">

<img src="tmp_2210.17434/./Figures/ami_psf.png" alt="Fig1" width="100%"/>

**Figure 1. -** The JWST-NIRISS AMI NRM, PSF, and its uv-coverage. \textbf{Left}: AMI's 7 hole pupil mask. \textbf{Middle}: a point source image taken through the NRM, showing a bright peak at the phase center, and with the primary beam crisscrossed by fringe patterns at 21 spatial frequencies. \textbf{Right}: The Fourier amplitude of the PSF. Note 21 "splodges" isolating regions of strong fringe signal (in red) and their mirrored Hermitian counterparts in this $\uvplane$ coverage, or Modulation Transfer Function (MTF), plot. (*fig:pup_psf_mtf*)

</div>
<div id="div_fig2">

<img src="tmp_2210.17434/./Figures/NRMprototype.png" alt="Fig2.1" width="50%"/><img src="tmp_2210.17434/./Figures/CLEARP.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Left: A full size prototype of the NRM flying aboard NIRISS AMI.  The nominal JWST pupil circumscribing circle diameter is  $40 \mathrm{mm}$ in the plane of the mask.  Right: The CLEARP pupil mask enables full pupil images using AMI filters, target acquisition, and fast readout subarray. The backwards-facing reflective element at the center of CLEARP is an unilluminated pupil alignment mirror which defines the inner edge of the pupil.  This reflective surface represents 8.8\% of the full telescope pupil, while the telescope central obscuration is $\sim$5.2\%.   Thus some scattered light from starlight is possible from the edges of this pupil alignment mirror.   CLEARP’s outer edge is oversized by $\sim$4\% compared to the JWST primary mirror. (*fig:g7s6clearp*)

</div>
<div id="div_fig3">

<img src="tmp_2210.17434/./Figures/zodi_visamps_com_sim_rc.png" alt="Fig3" width="100%"/>

**Figure 3. -** Squared visibilities as a function of maximum baseline length. Simulated AMI observations of the dusty debris system $\eta$ Crv  and a hypothetical exozodiacal system with 100 times lower dust density were analyzed with a precursor to the \texttt{AMICAL} interferometric package.  Commissioning results (utilizing \texttt{ImPlaneIA}) of a calibrator star calibrated against another calibrator star are also shown.  These data indicate $\eta$ Crv's zodiacal light can be detected with AMI.  Improved charge migration calibration could yet render the fainter disk detectable. Such data could constrain the form of the extrazodiacal dust distribution in a routine AMI observation of a single target-calibrator pair.  JWST Program 1242 includes such an observation. (*fig:zodi*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

74  publications in the last 7 days.
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers